In [2]:
import pandas as pd
import numpy as np


df = pd.read_csv("pokemon_data/pokemon.csv")
ptype_df = pd.read_csv("pokemon_data/pokemon_type_matchup.csv")

def get_typelist(typedata):
    if "–" in typedata:
        return []
    typelist = typedata.split(", ")
    typelist = [s.lower() for s in typelist if (not "(no effect)" in s)]
    return typelist

def get_noeffectlist(typedata):
    noeffectlist = typedata.split(", ")
    noeffectlist = [s.lower() for s in noeffectlist if ("(no effect)" in s)]
    noeffectlist = [s.replace(" (no effect)", "") for s in noeffectlist]
    return noeffectlist

def matchup(types1, types2):
    type1_attack_score = 0
    type1_def_score = 0
    
    for type1 in types1:
        if type1 is np.nan:
                continue
        type1_data = ptype_df[ptype_df["type"] == type1]
        for type2 in types2:
            if type2 is np.nan:
                continue

            #type2_data = ptype_df[ptype_df["type"] == type2]
            type1_attack_score += len([
                ptype for ptype in type1_data["strong_against"].values[0] if ptype == type2
            ])
            type1_attack_score -= len([
                ptype for ptype in type1_data["weak_against"].values[0]if ptype == type2
            ])
            type1_attack_score -= 1000*len([
                ptype for ptype in type1_data["att_no_effect"].values[0] if ptype == type2
            ])

            type1_def_score += len([
                ptype for ptype in type1_data["resistant_to"].values[0] if ptype == type2
            ])
            type1_def_score -= len([
                ptype for ptype in type1_data["weak_to"].values[0] if ptype == type2
            ])
            type1_def_score += 1000*len([
                ptype for ptype in type1_data["def_no_effect"].values[0] if ptype == type2
            ])
        #print(f"{type1} {type2} {type1_attack_score} {type1_def_score}")

    return type1_attack_score, type1_def_score
    

def get_standings_optimized(df_gen, pokemon):
    row = df_gen["name"] == pokemon
    types = df_gen[row]["type1"].values[0], df_gen[row]["type2"].values[0]


    att_wins = lambda matchup_entry: matchup_entry[0] > 0
    def_wins = lambda matchup_entry: matchup_entry[1] > 0

    att_draws = lambda matchup_entry: matchup_entry[0] == 0
    def_draws = lambda matchup_entry: matchup_entry[1] == 0

    att_losses = lambda matchup_entry: matchup_entry[0] < 0
    def_losses = lambda matchup_entry: matchup_entry[1] < 0
    
    matchups = df_gen["name"].apply(
        lambda pokemon2: (lambda pokemon2_entry: matchup(types, [pokemon2_entry["type1"].values[0], pokemon2_entry["type2"].values[0]]))(df_gen[df_gen["name"] == pokemon2])
    )
    return [sum(matchups.apply(scoring_function)) for scoring_function in [
        att_wins, att_draws, att_losses, def_wins, def_draws, def_losses
    ]]


def main(ptype_df=ptype_df, gen=3):
    
    # Get generation-specific df
    df_gen = df[df["generation"] <= gen] # gen 3 and before, not just gen 3
    df_gen["name"] = df_gen["name"].apply(lambda name: name.lower())

    ptype_df.columns = "type strong_against weak_against resistant_to weak_to".split()

    # Get noeffect data
    ptype_df["att_no_effect"] = ptype_df["weak_against"].apply(get_noeffectlist)
    ptype_df["def_no_effect"] = ptype_df["resistant_to"].apply(get_noeffectlist)


    # Format all columns
    ptype_df["type"] = ptype_df["type"].apply(lambda s: s.lower())

    for column_name in ptype_df.columns[1:-2]:
        if not (type(ptype_df[column_name].to_list()[0]) == list):
            ptype_df[column_name] = ptype_df[column_name].apply(get_typelist)


    all_standings_optimized = df_gen["name"].apply(lambda name: (name, get_standings_optimized(name)))

    # sorted(all_standings_optimized.to_list(), key=lambda v: -v[1][0])

    data = all_standings_optimized.to_list()
    columns = ["name", "att_wins", "att_draws", "att_losses", "def_wins", "def_draws", "def_losses"]

    standings_df = pd.DataFrame([[x[0]] + x[1] for x in data], columns=columns) 
    standings_df.head()

    standings_df.sort_values(by="def_losses", ascending=False)
    
    standings_df.to_csv("standings.csv")

main()

C:\Users\durbi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def generate_data(df, ptype_df, gen=3):
    